In [106]:
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd

In [107]:
code_df = pd.read_csv("data/stock_code/codes.csv")

In [108]:
code_df.head()

,コード,銘柄名,市場区分,本社所在地,業務分類,決算期,売買単位,支配株主等に関する事項,注意情報等,継続企業の前提の注記
0,13010,極洋,第一部,東京,水産・農林業,3月,100,無し,,無し
1,13050,ダイワ上場投信－トピックス,ETF,-,-,-,10,-,,-
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF,-,-,-,10,-,,-
3,13080,上場インデックスファンドＴＯＰＩＸ,ETF,-,-,-,100,-,,-
4,13090,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF,-,-,-,1,-,,-


In [109]:
code_df["code"] = (code_df["コード"]/10).astype(int).astype(str)
code_df["yahoo_code"] = code_df.code+".T"

In [110]:
code_df.head()

,コード,銘柄名,市場区分,本社所在地,業務分類,決算期,売買単位,支配株主等に関する事項,注意情報等,継続企業の前提の注記,code,yahoo_code
0,13010,極洋,第一部,東京,水産・農林業,3月,100,無し,,無し,1301,1301.T
1,13050,ダイワ上場投信－トピックス,ETF,-,-,-,10,-,,-,1305,1305.T
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF,-,-,-,10,-,,-,1306,1306.T
3,13080,上場インデックスファンドＴＯＰＩＸ,ETF,-,-,-,100,-,,-,1308,1308.T
4,13090,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF,-,-,-,1,-,,-,1309,1309.T


In [111]:
import datetime
import time
sdf = None
for i,cr in code_df.iterrows():
  yc = cr.yahoo_code
  print(yc)
  my_share = share.Share(yc)
  symbol_data = None
  try:
    symbol_data = my_share.get_historical(
                  share.PERIOD_TYPE_DAY, 10,
                  share.FREQUENCY_TYPE_DAY, 1)
  except YahooFinanceError as e:
    print("{}:{},{}".format(e.message,yc,cr["銘柄名"]))
    continue

  if symbol_data is None:
    print("{}/{} is None".format(yc,cr["銘柄名"]))
    continue

  tmpdf = pd.DataFrame(symbol_data)
  tmpdf["datetime"] = pd.to_datetime(tmpdf.timestamp, unit="ms")
  tmpdf["code_y"] = yc
  tmpdf["code"] = yc[0:4]
  ordered_tmp = tmpdf[["code","datetime","open","high","low","close","code_y"]]
  if sdf is None:
    sdf = ordered_tmp
  else:
    sdf = sdf.append(ordered_tmp)
    sdf.to_csv("data/transaction_data/transaction.csv",index=False)
  time.sleep(1)
#  if i > 10:
#     break

print(len(sdf))

1301.T
1305.T
1306.T
1308.T
1309.T
1311.T
1312.T
1313.T
1319.T
1320.T
1321.T
1322.T
1323.T
1324.T
1325.T
1326.T
1327.T
1328.T
1329.T
1330.T
1332.T
1333.T
1343.T
1344.T
1345.T
1346.T
1348.T
1349.T
1352.T
1356.T
1357.T
1358.T
1360.T
1364.T
1365.T
1366.T
1367.T
1368.T
1369.T
1375.T
1376.T
1377.T
1379.T
1380.T
1381.T
1382.T
1383.T
1384.T
1385.T
1386.T
1387.T
1388.T
1389.T
1390.T
1391.T
1392.T
1393.T
1394.T
1397.T
1398.T
1399.T
1400.T
1401.T
1407.T
1413.T
1414.T
1417.T
1418.T
1419.T
1420.T
1429.T
1430.T
1431.T
1432.T
1433.T
1434.T
1435.T
1436.T
1439.T
1440.T
1443.T
1445.T
1446.T
1447.T
1448.T
1450.T
1451.T
1452.T
1456.T
1457.T
1458.T
1459.T
1460.T
1464.T
1465.T
1466.T
1467.T
1468.T
1469.T
1471.T
1472.T
1473.T
1474.T
1475.T
1476.T
1477.T
1478.T
1479.T
1480.T
1481.T
1482.T
1483.T
1484.T
1485.T
1486.T
1487.T
1488.T
1489.T
1490.T
1491.T
1492.T
1493.T
1494.T
1495.T
1496.T
1497.T
1498.T
1499.T
1514.T
1515.T
1518.T
1540.T
1541.T
1542.T
1543.T
1545.T
1546.T
1547.T
1550.T
1551.T
1552.T
1554.T
1555.T

AttributeError: 'DataFrame' object has no attribute 'timestamp'

In [ ]:
sdf.head()

,code,datetime,open,high,low,close,code_y
0,1301,2021-07-19 00:00:00,2948.0,2948.0,2908.0,2917.0,1301.T
1,1301,2021-07-20 00:00:00,2917.0,2919.0,2903.0,2909.0,1301.T
2,1301,2021-07-21 00:00:00,2930.0,2960.0,2930.0,2954.0,1301.T
3,1301,2021-07-26 00:00:00,2977.0,2988.0,2956.0,2963.0,1301.T
4,1301,2021-07-27 06:15:01,2972.0,2988.0,2967.0,2981.0,1301.T


In [ ]:
sdf.to_csv("data/transaction_data/transaction.csv",index=False)